In [1]:
import pandas as pd
import psycopg2
import boto3
import json

### SECRETS MANAGER 

___

Una vez loggeado en la consola de AWS ir al **secrets manager** para conocer los secretos necesarios para conectarse a la isntacia RDS y consultar la BD

En el **secrets manager** habra que ingresar a *postgres-credentials* y ahi clickear el boton *Retrieve secret value*

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

host = os.getenv('RDS_HOST')
user = os.getenv('RDS_USER')
password = os.getenv('RDS_PASS')

In [3]:
conn = psycopg2.connect(
    host = host,
    database = 'postgres',
    user = user,
    password = password)
cursor = conn.cursor()

In [4]:
cursor.execute("""
        SELECT table_catalog, table_schema, table_name, table_type
        FROM information_schema.tables 
        WHERE table_schema = 'monks';
""")

results = pd.DataFrame(cursor.fetchall())
results.columns = [desc[0] for desc in cursor.description]
results

,table_catalog,table_schema,table_name,table_type
0,postgres,monks,hits,BASE TABLE
1,postgres,monks,daily_hits_agg,BASE TABLE


ambas tablas fueron creadas

In [5]:
cursor.execute("SELECT * FROM monks.hits LIMIT 10;")
results = pd.DataFrame(cursor.fetchall())
results.columns = [desc[0] for desc in cursor.description]
results

,userid,visitid,date,device,city,hit_count
0,884932399915132,1499010813,2017-07-02,mobile,San Francisco,9
1,1563469934876372,1498960250,2017-07-01,mobile,San Jose,6
2,1563469934876372,1498960593,2017-07-01,mobile,San Jose,2
3,3409332501133020,1498936817,2017-07-01,mobile,not available in demo dataset,1
4,4946235279474055,1498961332,2017-07-01,mobile,not available in demo dataset,1
5,8254883062403392,1498968649,2017-07-01,desktop,not available in demo dataset,1
6,12926685806111912,1499021725,2017-07-02,desktop,not available in demo dataset,1
7,14201933474364573,1498947269,2017-07-01,desktop,not available in demo dataset,2
8,16064855417596824,1498902310,2017-07-01,desktop,not available in demo dataset,4
9,17049423592725683,1498901840,2017-07-01,desktop,not available in demo dataset,9


tenemos el conteo de hits por user/session

In [6]:
cursor.execute("SELECT * FROM monks.daily_hits_agg;")
results = pd.DataFrame(cursor.fetchall())
results.columns = [desc[0] for desc in cursor.description]
results

,date,hits_total_count,users_total_count
0,2017-07-01,7975,1946
1,2017-07-02,6618,1791
2,2017-07-03,7705,1904
3,2017-07-04,6661,1799


los calculos agregados diariamente son correctos

In [7]:
cursor.close()
conn.close()

Corroborando calculos

In [9]:
df1=pd.read_json('./data/bquxjob_1178c782_183cbd27267.json')
df2=pd.read_json('./data/bquxjob_1c947e3c_183cbd23bb4.json')
df3=pd.read_json('./data/bquxjob_1fa5eddf_183cbd1de05.json')

df = pd.concat([df1,df2,df3], ignore_index=True)

df = df.explode('hit')

In [10]:
df.groupby("date")['userId'].nunique()

date
20170701    1946
20170702    1791
20170703    1904
20170704    1799
Name: userId, dtype: int64

In [12]:
df.groupby("date")['hit'].count()

date
20170701    7975
20170702    6618
20170703    7705
20170704    6661
Name: hit, dtype: int64

tanto el conteo de unique users por dia y el conteo de hits calculado por las funciones lambda y escrito en la BD RDS son correctos o al menos coinciden con los json files